In [1]:
## This script is developed for the purpose of visualization 
## of the FARGO output
## This script combines the FARGO recommended script and the multifluid script developed by me

# Author: sayantan
# Date : 14 August 2019
## modified : 26 August 2019 added the polar plots

from pylab import*
import matplotlib.pyplot as plt
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rc('text', usetex=True)
import os
import glob
import time
import multiprocessing as mp
import cv2

In [2]:
################################### User defined inputs ##################
## give to path to the directory of the output
# folder = '/Volumes/My_Passport/TIARA_DATA/21_Aug_Pablo_EQ/pablo_eq_setup/outputs/fargo_multifluid_2D_H/'
# folder = '/Volumes/My_Passport/TIARA_DATA/21_Aug_Pablo_EQ/pablo_eq_setup/outputs/fargo_multifluid_2D/'
# folder = '/Volumes/My_Passport/TIARA_DATA/21_Aug_Pablo_EQ/T1/outputs/fargo_multifluid_T1/'
# folder = '/Volumes/My_Passport/TIARA_DATA/21_Aug_Pablo_EQ/T2/outputs/fargo_multifluid_T2/'
# folder = '/Volumes/My_Passport/TIARA_DATA/21_Aug_Pablo_EQ/T4/outputs/fargo_multifluid_T4/'
folder = 'fargo_multifluid_T0/'
# folder = './T8/outputs/fargo_multifluid_T8/'
# folder = './T8_0/outputs/fargo_multifluid_T8_0/'
fluid_list = ['gas','dust1','dust2'] 
parameter_list = ['dens','vy','vx']
##please select the particular fluid and the parameter from the list
f=1;p=0
fluid = fluid_list[f]
parameter= parameter_list[p]

## type of plot

# plot_type = "2D"
plot_type = "1D"
# plot_type = "2D_polar"

In [3]:
## Making the Parameter class 
class Parameters(object):
    """
    Class for reading the simulation parameters.
    input: string -> name of the parfile, normally variables.par
    """
    def __init__(self, paramfile):
        try:
            params = open(paramfile,'r') #Opening the parfile
        except IOError:                  # Error checker.
            print  (paramfile + " not found.")
            return
        lines = params.readlines()     # Reading the parfile
        params.close()                 # Closing the parfile
        par = {}                       # Allocating a dictionary
        for line in lines:             #Iterating over the parfile
            name, value = line.split() #Spliting the name and the value (first blank)
            try:
                float(value)           # First trying with float
            except ValueError:         # If it is not float
                try:
                    int(value)         #                   we try with integer
                except ValueError:     # If it is not integer, we know it is string
                    value = '"' + value + '"'
            par[name] = value          # Filling the dictory
        self._params = par             # A control atribute, actually not used, good for debbuging
        for name in par:               # Iterating over the dictionary
            exec("self."+name.lower()+"="+par[name]) #Making the atributes at runtime



In [4]:
P = Parameters(folder+"variables.par")
print (P.nx, P.ny, P.omegaframe, P.outputdir, P.sigmaslope, P.sigma0)
xmin = loadtxt(folder+"domain_x.dat")       #face-centered coodinates
ymin = loadtxt(folder+"domain_y.dat")[3:-3] #face-centered coordinates. Ghosts are included, so we remove them from the array
xc   = 0.5*(xmin[1:]+xmin[:-1]) #cell-centered
yc   = 0.5*(ymin[1:]+ymin[:-1]) #cell-centered
# P._params


384 128 0 ./outputs/fargo_multifluid_T0/ 0.5 0.0001


In [5]:
list_of_files = glob.glob(folder + fluid + parameter + '*.dat')
if (folder + fluid + parameter +'0_2d.dat') in list_of_files:
    list_of_files.remove(folder + fluid + parameter +'0_2d.dat')
    
list=  sorted(list_of_files, key=lambda x: int(x.replace(folder + fluid + parameter, '').replace('.dat', '')))
# print(list)
current_directory = os.getcwd() 
# print("the current directory in", current_directory)
path= current_directory + '/movie_outputs_'+ fluid +'_'+ parameter
path2 = current_directory + '/movie_folder'


# os.path.isdir(path)
try:      
    os.makedirs(path)
except OSError:  
    print ("Creation of the directory %s failed/ not needed as it already exit" % path)
else:  
    print ("Successfully created the directory %s" % path)
    
try:      
    os.makedirs(path2)
except OSError:  
    print ("Creation of the directory %s failed/ not needed as it already exit" % path)
else:  
    print ("Successfully created the directory %s" % path)

Creation of the directory /Users/sayantan/Desktop/Programming/Dusty_Disk_Gap/movie_outputs_dust1_dens failed/ not needed as it already exit
Creation of the directory /Users/sayantan/Desktop/Programming/Dusty_Disk_Gap/movie_outputs_dust1_dens failed/ not needed as it already exit


In [6]:
## This part of the code was introduced to plot and save the images using parallel processing
number_of_orbit = len(list)
# print(number_of_orbit)
# number_of_orbit = 10
start = time.time()
print("Plotting now...")
def do_plot(number):
    
    '''
    This part of the code is taken from Stackoverflow
    The plot function:
    Input: The length of the data file 
    Output: The images in the specified output folder 
    
    '''
    base_param = fromfile(list[0]).reshape(P.ny,P.nx)
    mean_base_param = base_param.mean(axis=1)
    filename = list[number]
#     print(filename)
    fig = plt.figure(number)
    param = fromfile(filename).reshape(P.ny,P.nx)
    mean_param=param.mean(axis=1)
    mean_variation_param = abs((mean_param-mean_base_param))/ mean_base_param
    
    if plot_type == "2D":
        imshow(log10(param),cmap=plt.cm.Spectral,origin='lower', aspect='auto',extent=[P.xmin,P.xmax,P.ymin,P.ymax])
        plt.title("Fluid = %s"%fluid + r" $\epsilon$= %s"%P.epsilon+ "  Orbit = %s"%number+ " res: %s"%P.ny +r"$\times$ %s"%P.nx, fontsize=12,color='Black')
        plt.ylabel(r"Radius", fontsize=15)
        plt.xlabel(r"$\varphi$")
        plt.colorbar()
      
    if plot_type == "2D_polar":
        r, theta = np.meshgrid(xc, yc)
#         fig = plt.figure()
        ax = plt.subplot(111, polar=True)
        ax.yaxis.grid(False)
        ax.xaxis.grid(False)
#         cax = ax.contourf(r, theta, log10(param), 100,cmap='Blues')  
        cax = ax.contourf(r, theta, log10(param), 100 , cmap=plt.cm.Spectral)
        cbar = fig.colorbar(cax)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        cbar.ax.set_ylabel(r"Log $\Sigma$")
        plt.title("Fluid = %s"%fluid + "  Orbit = %s"%number+ " res: %s"%P.ny +r"$\times$ %s"%P.nx, fontsize=12,color='Black')
       
    elif plot_type == "1D":
        plt.plot(yc,base_param[:,0],label= "Initial %s"%fluid + " Density" + " res: %s"%P.ny +r"$\times$ %s"%P.nx )  
        plt.plot(yc,param[:,0],linestyle='--',label= "Evolving %s"%fluid + r" density for a particular $\phi$" + " Orbit = %s"%number)  
#         plt.plot(yc,mean_param,linestyle='-',label= "Evolving %s"%fluid + " Density"+" Orbit = %s"%number) ## plotting the radial profile
#         plt.plot(yc,mean_variation_param,label="Orbit = %s"%number +" Fluid = %s"%fluid)       
#         plt.plot(yc,mean_base_param,label= "Initial %s"%fluid + " Density")        
#         plt.plot(yc,mean_param,linestyle='--',label= "Evolving %s"%fluid + " Density") 
        plt.axhline(y=1, color='r', linestyle='--')      
        plt.legend(loc=2)
        plt.yscale('log')

#         plt.text(50,4,"StockHolm Turned on",fontsize=12,color='red')

        plt.ylabel(r"Log $\Sigma$", fontsize=15)
#         plt.ylabel(r"Log $\Delta \Sigma/\Sigma_{0}$", fontsize=15)
        plt.xlabel(r"Radius", fontsize=15)
        plt.tight_layout()
    outputfilename_figfile = os.path.join(path, fluid +'_'+ parameter +'%s'%number+ ".jpg")
    plt.savefig(outputfilename_figfile,format='jpg',dpi=300)
    plt.close()
    
if __name__ == '__main__':
    pool = mp.Pool()
    pool.map(do_plot, range(0,number_of_orbit,10))
print("End of simulation")    
end = time.time()
T = end-start
print("Total time for parallel run %.2f"%T, 'sec')

Plotting now...
End of simulation
Total time for parallel run 4.24 sec


In [7]:
## For making the movie: 

print("Making the Video")

file = folder.split("multifluid_",1)[1].replace('/','')
image_folder = path 
video_name = os.path.join(path2, file+ '_video_'+ fluid +'_'+ parameter + '_' + plot_type + '.mov')
# video_name = file+'video_'+ fluid +'_'+ parameter + '_' + plot_type + '.mov'
if os.path.isfile(video_name):
    print("Removing the old video and making the new one")
    os.remove(video_name)

image = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
## sorting the images in ascending order of time
images= sorted(image, key=lambda x: int(x.replace(fluid +'_'+ parameter, '').replace('.jpg', '')))
# images =sorted(image)
# print(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, -1, 5, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

print("The video is done")

Making the Video
The video is done


In [8]:
## This is delete all the images to save space
fileList = os.listdir(path)

for fileName in fileList:
#     print(fileName)
    os.remove(path+"/"+fileName)


